* 请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

## 1.下载paddleX、跑通一个图像分类的示例
### 1.1下载paddle X

### 1.2跑通image classification示例
本次试用的示例为**果蔬分类**

<center><img src='https://ai-studio-static-online.cdn.bcebos.com/ac42cb4860b44496ad05ce3d3b37d661c4224843b756487dbb0a755373c468cd' width='700'></center>

## 2.分割任务实现

### 2.1数据集下载和解压
* 安装paddlex

In [1]:
!pip install paddlex

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 516 kB 12.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 15.9 MB/s eta 0:00:01
     |████████████████████████████████| 211 kB 18.2 MB/s eta 0:00:01
     |████████████████████████████████| 99 kB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 145 kB 31.2 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 3.4 MB/s  eta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278376 sha256=0d7d9e8c215a1bb49aff24c6dbdb80895f1bfcd26fc24de8de69dfb39ab13d11
  Stored in directory: /home/aistudio/.cache/pip/wheels/3b/1c/8f/488903e8b13ce4f1f6397a0da41ea9df58356d7a7b56c89e38
Successfully built pycocotools
  Attempting uninstall: paddlehub
    Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4
You should consider upgrading via the '/opt/con

* 下载和解压数据

In [2]:
import paddlex as pdx
optic_dataset = 'https://bj.bcebos.com/paddlex/datasets/optic_disc_seg.tar.gz'
pdx.utils.download_and_decompress(optic_dataset, path='./')

2021-05-12 09:48:46 [INFO]	Downloading optic_disc_seg.tar.gz from https://bj.bcebos.com/paddlex/datasets/optic_disc_seg.tar.gz


100%|██████████| 18631/18631 [00:00<00:00, 68924.22KB/s]


2021-05-12 09:48:47 [INFO]	Decompressing ./optic_disc_seg.tar.gz...


### 2.2 数据集划分

In [3]:
import paddlex
!paddlex --split_dataset --format SEG --dataset_dir optic_disc_seg --val_value 0.2 --test_value 0.1

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Dataset Split Done.
Train samples: 267
Eval samples: 76
Test samples: 38
Split files saved in optic_disc_seg


### 2.3 语义分割模型训练及保存

In [4]:
import paddlex as pdx
from paddlex.seg import transforms
# 定义训练和验证时的transforms

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(), transforms.ResizeRangeScaling(),
    transforms.RandomPaddingCrop(crop_size=512), transforms.Normalize()
])

eval_transforms = transforms.Compose([
    transforms.ResizeByLong(long_size=512),
    transforms.Padding(target_size=512), transforms.Normalize()
])

# 定义训练和验证所用的数据集

train_dataset = pdx.datasets.SegDataset(
    data_dir='optic_disc_seg',
    file_list='optic_disc_seg/train_list.txt',
    label_list='optic_disc_seg/labels.txt',
    transforms=train_transforms,
    shuffle=True)

eval_dataset = pdx.datasets.SegDataset(
    data_dir='optic_disc_seg',
    file_list='optic_disc_seg/val_list.txt',
    label_list='optic_disc_seg/labels.txt',
    transforms=eval_transforms)

# 初始化模型，并进行训练

num_classes = len(train_dataset.labels)

model = pdx.seg.DeepLabv3p(
    num_classes=num_classes,
    backbone='MobileNetV3_large_x1_0_ssld',
    pooling_crop_size=(512, 512))


model.train(
    num_epochs=10,
    train_dataset=train_dataset,
    train_batch_size=4,
    eval_dataset=eval_dataset,
    learning_rate=0.01,
    save_dir='output/deeplabv3p_mobilenetv3_large_ssld',
    use_vdl=True)

2021-05-12 09:48:49 [INFO]	267 samples in file optic_disc_seg/train_list.txt
2021-05-12 09:48:49 [INFO]	76 samples in file optic_disc_seg/val_list.txt


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/mobilenet_v3.py:231
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_mul(X, Y, axis=0) instead of A * B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/segmentation/deeplabv3p.py:287
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elemen

2021-05-12 09:48:52 [INFO]	Connecting PaddleHub server to get pretrain weights...
Download https://paddle-imagenet-models-name.bj.bcebos.com/MobileNetV3_large_x1_0_ssld_pretrained.tar
[##################################################] 100.00%
Decompress /home/aistudio/.paddlehub/tmp/tmpik0ujkzl/MobileNetV3_large_x1_0_ssld_pretrained.tar
[##################################################] 100.00%
2021-05-12 09:48:53 [INFO]	Load pretrain weights from output/deeplabv3p_mobilenetv3_large_ssld/pretrain/MobileNetV3_large_x1_0_ssld.
2021-05-12 09:48:53 [WARNING]	[SKIP] Shape of pretrained weight output/deeplabv3p_mobilenetv3_large_ssld/pretrain/MobileNetV3_large_x1_0_ssld/conv14_expand_weights doesn't match.(Pretrained: (672, 112, 1, 1), Actual: (336, 112, 1, 1))
2021-05-12 09:48:53 [WARNING]	[SKIP] Shape of pretrained weight output/deeplabv3p_mobilenetv3_large_ssld/pretrain/MobileNetV3_large_x1_0_ssld/conv14_expand_bn_scale doesn't match.(Pretrained: (672,), Actual: (336,))
2021-05-12 09:

100%|██████████| 19/19 [00:17<00:00,  1.10it/s]


2021-05-12 09:51:21 [INFO]	[EVAL] Finished, Epoch=1, miou=0.688527, category_iou=[0.98898135 0.38807266], oacc=0.989058, category_acc=[0.98918949 0.97076879], kappa=0.55462, category_F1-score=[0.99446015 0.55915324] .
2021-05-12 09:51:21 [INFO]	Model saved in output/deeplabv3p_mobilenetv3_large_ssld/best_model.
2021-05-12 09:51:21 [INFO]	Model saved in output/deeplabv3p_mobilenetv3_large_ssld/epoch_1.
2021-05-12 09:51:21 [INFO]	Current evaluated best model in eval_dataset is epoch_1, miou=0.6885270044086613
2021-05-12 09:51:40 [INFO]	[TRAIN] Epoch=2/10, Step=2/66, loss=0.024293, lr=0.009082, time_each_step=2.48s, eta=0:22:40
2021-05-12 09:51:44 [INFO]	[TRAIN] Epoch=2/10, Step=4/66, loss=0.044966, lr=0.009054, time_each_step=2.51s, eta=0:22:37
2021-05-12 09:51:48 [INFO]	[TRAIN] Epoch=2/10, Step=6/66, loss=0.016444, lr=0.009026, time_each_step=2.57s, eta=0:22:35
2021-05-12 09:51:53 [INFO]	[TRAIN] Epoch=2/10, Step=8/66, loss=0.0153, lr=0.008999, time_each_step=2.61s, eta=0:22:33
2021-05-1

...

2021-05-12 10:08:40 [INFO]	[EVAL] Finished, Epoch=8, miou=0.873558, category_iou=[0.99514999 0.75196696], oacc=0.99522, category_acc=[0.99676623 0.90067703], kappa=0.856, category_F1-score=[0.9975691  0.85842596] .
2021-05-12 10:08:40 [INFO]	Model saved in output/deeplabv3p_mobilenetv3_large_ssld/best_model.
2021-05-12 10:08:40 [INFO]	Model saved in output/deeplabv3p_mobilenetv3_large_ssld/epoch_8.
2021-05-12 10:08:40 [INFO]	Current evaluated best model in eval_dataset is epoch_8, miou=0.8735584732839974
2021-05-12 10:08:56 [INFO]	[TRAIN] Epoch=9/10, Step=2/66, loss=0.014727, lr=0.002333, time_each_step=2.34s, eta=0:5:14
2021-05-12 10:09:01 [INFO]	[TRAIN] Epoch=9/10, Step=4/66, loss=0.014705, lr=0.002301, time_each_step=2.38s, eta=0:5:12
2021-05-12 10:09:05 [INFO]	[TRAIN] Epoch=9/10, Step=6/66, loss=0.009245, lr=0.002269, time_each_step=2.42s, eta=0:5:10
2021-05-12 10:09:09 [INFO]	[TRAIN] Epoch=9/10, Step=8/66, loss=0.013425, lr=0.002237, time_each_step=2.45s, eta=0:5:7
2021-05-12 10:0

100%|██████████| 19/19 [00:16<00:00,  1.14it/s]


2021-05-12 10:11:07 [INFO]	[EVAL] Finished, Epoch=9, miou=0.878042, category_iou=[0.99528654 0.76079835], oacc=0.995356, category_acc=[0.9970488  0.89453528], kappa=0.861792, category_F1-score=[0.9976377  0.86415159] .
2021-05-12 10:11:07 [INFO]	Model saved in output/deeplabv3p_mobilenetv3_large_ssld/best_model.
2021-05-12 10:11:08 [INFO]	Model saved in output/deeplabv3p_mobilenetv3_large_ssld/epoch_9.
2021-05-12 10:11:08 [INFO]	Current evaluated best model in eval_dataset is epoch_9, miou=0.878042443418493
2021-05-12 10:11:24 [INFO]	[TRAIN] Epoch=10/10, Step=2/66, loss=0.013437, lr=0.001242, time_each_step=2.35s, eta=0:2:48
2021-05-12 10:11:29 [INFO]	[TRAIN] Epoch=10/10, Step=4/66, loss=0.011733, lr=0.001207, time_each_step=2.39s, eta=0:2:46
2021-05-12 10:11:33 [INFO]	[TRAIN] Epoch=10/10, Step=6/66, loss=0.011967, lr=0.001173, time_each_step=2.42s, eta=0:2:43
2021-05-12 10:11:37 [INFO]	[TRAIN] Epoch=10/10, Step=8/66, loss=0.018182, lr=0.001138, time_each_step=2.46s, eta=0:2:40
2021-05

100%|██████████| 19/19 [00:16<00:00,  1.14it/s]


2021-05-12 10:13:33 [INFO]	[EVAL] Finished, Epoch=10, miou=0.877277, category_iou=[0.99527721 0.75927725], oacc=0.995347, category_acc=[0.99695548 0.89846722], kappa=0.860806, category_F1-score=[0.99763301 0.86316952] .
2021-05-12 10:13:33 [INFO]	Model saved in output/deeplabv3p_mobilenetv3_large_ssld/epoch_10.
2021-05-12 10:13:33 [INFO]	Current evaluated best model in eval_dataset is epoch_9, miou=0.878042443418493


### 2.4 模型预测

In [5]:
#从测试数据中随机挑选一张图
test_jpg = 'optic_disc_seg/JPEGImages/P0099.jpg'
#下载训练好的模型
model = pdx.load_model('output/deeplabv3p_mobilenetv3_large_ssld/epoch_10')
result = model.predict(test_jpg)
pdx.seg.visualize(test_jpg, result, weight=0.0, save_dir='./')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/mobilenet_v3.py:231
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_mul(X, Y, axis=0) instead of A * B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/segmentation/deeplabv3p.py:287
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elemen

2021-05-12 10:13:34 [INFO]	Model[DeepLabv3p] loaded.
2021-05-12 10:13:34 [INFO]	The visualized result is saved as ./visualize_P0099.jpg


### 2.5 模型导出
导出模型保存在`./inference_model路径下`

In [6]:
!paddlex --export_inference --model_dir='output/deeplabv3p_mobilenetv3_large_ssld/epoch_10' --save_dir='./inference_model'

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/mobilenet_v3.py:231
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_mul(X, Y, axis=0) instead of A * B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/n